In [2]:
import pmagpy
import pmagpy.pmagplotlib as pmagplotlib
import pmagpy.ipmag as ipmag
import pmagpy.pmag as pmag
import pmagpy.contribution_builder as cb
from pmagpy import convert_2_magic as convert
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import os
from pathlib import Path
import vgptools
%matplotlib inline

In [3]:
home = Path(os.getcwd())
data_folder = home/'data'      # Directory for data files (Excel or csv) from publications
magic_folder = home/'magic'    # Directory for MagIC output

## Letts et al. 2009
Letts, S., Torsvik, T. H., Webb, S. J., & Ashwal, L. D. (2009). Palaeomagnetism of the 2054 Ma Bushveld Complex (South Africa): implications for emplacement and cooling. Geophysical Journal International, 179(2), 850–872. https://doi.org/10.1111/j.1365-246X.2009.04346.x

In [4]:
## Letts et al (2009)
#%%capture sites_conversion_log
# Capture output to sites_conversion_log

## Convert Sites XLSX Files to MagIC format

# In this XLSX file, here is how I interpret the mapping of relevant columns to the MagIC 3.0 data model (PAS 8/16/2024): 
#  
#
#  Region: sites.location (Note: there are also 'locations' for each )
#  DecLong: sites.lon
#  DecLat: sites.lat
#  n (col. L): sites.dir_n_total_samples (Note: as per conversation with Jeff and Rachel 10/15/2024, this may not be correct!)
#  Full Site: sites.site (had to add this column to remove ambiguities in site numbers)
#  GDec: sites.dir_dec
#  GInc: sites.dir_inc
#  Strike: sites.bed_dip_direction (=(strike+90.)%360.)
#  Dip: sites.bed_dip_direction
#  n (col. W): sites.dir_n_samples (Note: as per conversation with Jeff and Rachel 10/15/2024, this may not be correct!)
#  a95 (col. X): sites.dir_alpha95
#  kappa: sites.dir_k
#  Lithology: sites.lithologies
#  
# Assumptions:
#
#  sites.formation = "Bushveld Complex"
#  sites.geologic_classes = "Igenous:Intrusive"
#  sites.geologic_types = "Layered Inntrusion"
#  sites.result_type = "i"
#  sites.method_codes = "DE-BFL:DE-K:LP-DIR-AF:LP-DIR-T"
#  sites.citations = "10.1111/j.1365-246X.2009.04346.x"
#  sites.age = 2054
#  sites.age_unit = "Ma"
#  sites.dir_tilt_correction = 0 (geographc coordinates)
#  sites.result_quality = "g" if n (col. W) is not nan, otherwise "b"

letts_data = pd.read_excel(data_folder/'Letts_Bushveld.xlsx',sheet_name='Sheet1')
#display(letts_data)

In [5]:
# Compile sites dataframe
sites_df=pd.DataFrame([])
sites_df['site']=letts_data['Full Site'].astype('string')
sites_df['location']=letts_data['Region']
sites_df['lat']=letts_data['DecLat']
sites_df['lat']=sites_df['lat'].round(2)
sites_df['lon']=letts_data['DecLong']
sites_df['lon']=sites_df['lon'].round(2)
sites_df['dir_n_total_samples']=letts_data['n']
sites_df['dir_n_samples']=letts_data['n.1']
sites_df['dir_dec']=letts_data['GDec']
sites_df['dir_inc']=letts_data['GInc']
sites_df['bed_dip']=letts_data['Dip']
sites_df['bed_dip_direction']=vgptools.strike_to_dip_direction(letts_data,'Strike')
sites_df['dir_alpha95']=letts_data['a95.1']
sites_df['dir_k']=letts_data['kappa']
sites_df['lithologies']= letts_data['Lithology']
sites_df['formation']= "Bushveld Complex"
sites_df['geologic_classes']='Igneous:Intrusive'
sites_df['result_type']= "i"
sites_df['method_codes']= "DE-BFL:DE-K:LP-DIR-AF:LP-DIR-T"
sites_df['citations']= "10.1111/j.1365-246X.2009.04346.x"
sites_df['geologic_types']="Layered Intrusion"
sites_df['age']= 2054
sites_df['age_unit']= "Ma"
sites_df['dir_tilt_correction']=0
sites_df['result_quality']=vgptools.result_quality_criterion(letts_data,'n.1')

# Uncomment the next line to see the whole dataframe
#with pd.option_context('display.max_rows', 1000, 'display.max_columns', 1000):
    #display(sites_df)

In [6]:
# create tilt corrected dataframe with pmagpy.pmag.dotilt
sites_tilt_df = vgptools.tilt_correct(sites_df)

In [7]:
# Calculate and attach VGPs with tilt corrections
sites_tilt_df = vgptools.calculate_vgps(sites_tilt_df)

#with pd.option_context('display.max_rows', 1000, 'display.max_columns', 1000):
#    display(sites_tilt_df)

In [8]:
# In this block and the next, we create a "locations.txt" file by averaging sites for two different definitions of "location": 
#  Averaging by the "location" column (i.e. Main Zone Eastern Lobe, etc.)
#  Averaging over the entire complex
# These are initially created as two dataframes that are then merged in the next block.

# Averaging by location:
# Mapping names of columns. We will average site-level data. 
# Each site has a number of samples listed in the sites table, so we will use that as our N in the parametric bootstrap.
# Compile locations dataframe
locations_df1=vgptools.roll_up_sites_locations(sites_tilt_df,location_parameter='location',nb=0)
locations_df1['location_type']='Region'
locations_df1['geologic_classes']='Igneous:Intrusive'
locations_df1['age']=2054
locations_df1['age_unit']="Ma"

# Averaging for the whole complex:
locations_df2=vgptools.roll_up_sites_locations(sites_tilt_df,location_parameter='formation',nb=0)
locations_df2['location_type']='Region'
locations_df2['geologic_classes']='Igneous:Intrusive'
locations_df2['age']=2054
locations_df2['age_unit']="Ma"


In [9]:
display(locations_df2)
display(locations_df1)

,location,sites,lithologies,lat_s,lat_n,lon_e,lon_w,formation,dir_dec,dir_inc,dir_k,dir_n_samples,dir_alpha95,result_type,location_type,geologic_classes,age,age_unit
0,Bushveld Complex,MZW-10:MZW-11:MZW-12:MZW-13:MZW-14:MZW-15:MZW-...,Gabbronorite:Norite:Intrusives:Gabbro:Pyroxenite,-25.7,-23.4,30.1,27.2,Bushveld Complex,5.0,65.0,76.8,95,1.7,a,Region,Igneous:Intrusive,2054,Ma


,location,sites,lithologies,lat_s,lat_n,lon_e,lon_w,dir_dec,dir_inc,dir_k,dir_n_samples,dir_alpha95,result_type,location_type,geologic_classes,age,age_unit
0,Critical Zone,CZ-4:CZ-66:CZ-67:CZ-68:CZ-M:CZ-B1:CZ-B2:CZ-H2:...,Norite:Intrusives,-24.9,-24.4,30.1,27.3,1.7,61.7,66.4,8,6.8,a,Region,Igneous:Intrusive,2054,Ma
1,Main Zone Eastern Lobe,MZE-1:MZE-2a:MZE-2b:MZE-3:MZE-8:MZE-42:MZE-43:...,Gabbronorite:Norite,-25.7,-24.3,30.0,29.4,14.7,63.8,75.8,26,3.3,a,Region,Igneous:Intrusive,2054,Ma
2,Main Zone Northern Lobe,MZN-1:MZN-2:MZN-3:MZN-4:MZN-5:MZN-6:MZN-7:MZN-...,Intrusives,-24.2,-23.4,29.0,28.1,359.8,66.1,67.6,22,3.8,a,Region,Igneous:Intrusive,2054,Ma
3,Main Zone Western Lobe,MZW-10:MZW-11:MZW-12:MZW-13:MZW-14:MZW-15:MZW-...,Gabbronorite,-25.6,-25.5,28.2,27.3,3.2,66.5,180.8,31,1.9,a,Region,Igneous:Intrusive,2054,Ma
4,Upper Zone,UZ-5:UZ-9:UZ-39:UZ-40:UZ-57:UZ-60:UZ-64:UZ-70,Gabbro:Gabbronorite:Pyroxenite,-25.4,-24.4,29.9,27.2,357.0,62.1,42.6,8,8.6,a,Region,Igneous:Intrusive,2054,Ma


In [10]:
# Put the two dataframes (with different definitions of what a "location" is)
locations_df = pd.concat([locations_df1,locations_df2],axis=0,ignore_index=True)


# For optional additional notes, add description column to DF.

descr = [''] * len(locations_df['location'])

# Note that authors of publication chose to flip the data in row 4: upper zone, while we left them as is.
descr[4] = 'Choice made to flip this polarity in Letts, table 8'

locations_df['description']=descr
display(locations_df)




,location,sites,lithologies,lat_s,lat_n,lon_e,lon_w,dir_dec,dir_inc,dir_k,dir_n_samples,dir_alpha95,result_type,location_type,geologic_classes,age,age_unit,formation,description
0,Critical Zone,CZ-4:CZ-66:CZ-67:CZ-68:CZ-M:CZ-B1:CZ-B2:CZ-H2:...,Norite:Intrusives,-24.9,-24.4,30.1,27.3,1.7,61.7,66.4,8,6.8,a,Region,Igneous:Intrusive,2054,Ma,NaN,
1,Main Zone Eastern Lobe,MZE-1:MZE-2a:MZE-2b:MZE-3:MZE-8:MZE-42:MZE-43:...,Gabbronorite:Norite,-25.7,-24.3,30.0,29.4,14.7,63.8,75.8,26,3.3,a,Region,Igneous:Intrusive,2054,Ma,NaN,
2,Main Zone Northern Lobe,MZN-1:MZN-2:MZN-3:MZN-4:MZN-5:MZN-6:MZN-7:MZN-...,Intrusives,-24.2,-23.4,29.0,28.1,359.8,66.1,67.6,22,3.8,a,Region,Igneous:Intrusive,2054,Ma,NaN,
3,Main Zone Western Lobe,MZW-10:MZW-11:MZW-12:MZW-13:MZW-14:MZW-15:MZW-...,Gabbronorite,-25.6,-25.5,28.2,27.3,3.2,66.5,180.8,31,1.9,a,Region,Igneous:Intrusive,2054,Ma,NaN,
4,Upper Zone,UZ-5:UZ-9:UZ-39:UZ-40:UZ-57:UZ-60:UZ-64:UZ-70,Gabbro:Gabbronorite:Pyroxenite,-25.4,-24.4,29.9,27.2,357.0,62.1,42.6,8,8.6,a,Region,Igneous:Intrusive,2054,Ma,NaN,"Choice made to flip this polarity in Letts, ta..."
5,Bushveld Complex,MZW-10:MZW-11:MZW-12:MZW-13:MZW-14:MZW-15:MZW-...,Gabbronorite:Norite:Intrusives:Gabbro:Pyroxenite,-25.7,-23.4,30.1,27.2,5.0,65.0,76.8,95,1.7,a,Region,Igneous:Intrusive,2054,Ma,Bushveld Complex,


In [11]:
# Compile contributions dataframe
# Note that MagIC somehow does not recognize this when you upload it, so you have to specify the DOI and lab by hand anyway.
contribution_df=pd.DataFrame({'reference':["10.1111/j.1365-246X.2009.04346.x"],'lab_names':["Not Specified"]},index=[0])
print(contribution_df)

                          reference      lab_names
0  10.1111/j.1365-246X.2009.04346.x  Not Specified


In [12]:
# Changing dataframes to dictionaries for export
contribution_dicts=contribution_df.fillna('').to_dict('records')
locations_dicts=locations_df.fillna('').to_dict('records')
sites_dicts = sites_tilt_df.fillna('').to_dict('records')

In [13]:
# Write files for uploading to MagIC
# You will still need to upload these to a private contribution by hand
pmag.magic_write(magic_folder/'letts2009_contribution.txt', contribution_dicts, 'contribution')
pmag.magic_write(str(magic_folder/'letts2009_locations.txt'), locations_dicts, 'locations')
pmag.magic_write(magic_folder/'letts2009_sites.txt', sites_dicts, 'sites')

1  records written to file  C:\Users\paselkin\Dropbox\Research\intrusion_magnetics\published-data-to-magic\magic\letts2009_contribution.txt
6  records written to file  C:\Users\paselkin\Dropbox\Research\intrusion_magnetics\published-data-to-magic\magic\letts2009_locations.txt
101  records written to file  C:\Users\paselkin\Dropbox\Research\intrusion_magnetics\published-data-to-magic\magic\letts2009_sites.txt


(True,
 WindowsPath('C:/Users/paselkin/Dropbox/Research/intrusion_magnetics/published-data-to-magic/magic/letts2009_sites.txt'))